##### COORESPONDANCE DES FICHIERS COLLECTES

In [151]:
import pandas as pd
from datetime import datetime, timedelta
import os, sys

In [152]:
panier_df = pd.read_excel("Echantillon_Data_Scrapping_09032024_valide_code.xlsx")
panier_df.head()

,Date_de_collecte,Code_site,Libelle_du_produit,Code produit,Quantite,Prix_du_produit,Caracteristique,Unite,Unite_monetaire,Code,Observation
0,2024-03-09 04:02:44,adjovan,HARICOT BLANC VRAC [300G],1010703020301,300,500,Pas de Caracteristique,G,CFA,111111,Ce produit figure dans le panier de bien
1,2024-03-09 04:05:09,adjovan,CARTON DE COTE DE PORC (STERNUM) [10KG],1010203010201,10,13.000 12.000,Pas de Caracteristique,KG,CFA,111111,ce produit figure dans le panier de bien
2,2024-03-09 04:05:20,adjovan,COTES DE PORC SANS PEAU [500G],1010203010201,500,1.600,Le porc contient plusieurs nutriments essentie...,G,CFA,111111,ce produit figure dans le panier de bien
3,2024-03-09 04:05:23,adjovan,COTES/COTELETTES DE BOEUF FUMÉES [1/2KG],1010201010101,1,1.100,Pas de Caracteristique,KG,CFA,111111,ce produit figure dans le panier
4,2024-03-09 04:05:37,adjovan,ÉCHINE DE PORC [500G],1010203010201,500,1.500 1.000,"Tirée de la partie supérieure du cou, l’échine...",G,CFA,111111,ce produit figure dans le panier de bien


In [66]:
date1 = datetime.strptime("01-03-2024", "%d-%m-%Y")
date_str = date1.strftime("%d%m%Y")
fichier_journalier = os.path.join("Data_Collecte","mars",f"Data_Scrapping_{date_str}.xlsx")
df_journalier = pd.read_excel(fichier_journalier)
df_journalier.head()

,Code_site,Date_de_collecte,Libelle_du_produit,Quantite,Prix_du_produit,Unite_monetaire,Caracteristique,Unite
0,https://www.ivoirshop.ci/,2024-03-01,Toilettes portables pour enfants – 12mois à 6ans,Status:In stock,13.500,FCFA,NaN,NaN
1,https://www.ivoirshop.ci/,2024-03-01,Toilettes portables pour enfants – 12mois à 6ans,Status:In stock,13.500,FCFA,NaN,NaN
2,https://www.ivoirshop.ci/,2024-03-01,PHARES H7 LED S2,Status:In stock,13.000,FCFA,NaN,NaN
3,https://www.ivoirshop.ci/,2024-03-01,Ozone woofer/home cinema – (BT – USB – FM – TV...,Status:In stock,22.500,FCFA,NaN,NaN
4,https://www.ivoirshop.ci/,2024-03-01,Blender Incassable Binatone BLG – 555 – 1.5 L ...,Status:2 en stock,NaN,NaN,NaN,NaN


In [153]:
colonnes_a_comparer = ["Libelle_du_produit"]  

In [154]:
date_debut = datetime.strptime("01-03-2024", "%d-%m-%Y")  
date_fin = datetime.strptime("30-03-2024", "%d-%m-%Y")  

In [155]:
# Traiter les intervalles de prix pour obtenir le plus petit prix
def extract_min_price(value):
    if isinstance(value, str):
        if "-" in value:
            parts = value.split("-")
            if len(parts) >= 2:
                return int(parts[0].strip())
        elif " " in value:
            return int(value.split()[0])
    return value

while date_debut <= date_fin:
    date_str = date_debut.strftime("%d%m%Y")
    fichier_journalier = os.path.join("Data_Collecte","mars", f"Data_Scrapping_{date_str}.xlsx")
    try:
        df_journalier = pd.read_excel(fichier_journalier)
        
        lignes_correspondantes = pd.merge(df_journalier, panier_df[['Libelle_du_produit', 'Code produit', 'Code']], how="inner", on=colonnes_a_comparer)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].fillna("")
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(".", "") if isinstance(x, str) else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(",", "") if isinstance(x, str) else x)
        #lignes_correspondantes["Prix_du_produit"] = pd.to_numeric(lignes_correspondantes["Prix_du_produit"].str.replace("[^\d-]", "", regex=True), errors='coerce')
        #lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: float(x.split("-")[0]) if isinstance(x, str) and "-" in x else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(extract_min_price)
        
        fichier_sortie = os.path.join("Data_Valide","mars",f"Data_Scrapping_{date_str}.xlsx")
        lignes_correspondantes.to_excel(fichier_sortie, index=False)
        
    except FileNotFoundError:
        print(f"Le fichier {fichier_journalier} n'existe pas.")
 
    date_debut += timedelta(days=1)

Le fichier Data_Collecte\mars\Data_Scrapping_10032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_15032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_16032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_17032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_19032024.xlsx n'existe pas.
Le fichier Data_Collecte\mars\Data_Scrapping_24032024.xlsx n'existe pas.


##### ESTIMATION DES PRIX MANQUANTS

##### Fusion des différents fichiers journaliers

In [159]:
dossier = os.path.join("Data_Valide","mars")

dfs = []

for fichier in os.listdir(dossier):  
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)

fichier_sortie = os.path.join("Data_Valide","mars","Data_Scrapping_mars2024.xlsx")

df_final.to_excel(fichier_sortie, index=False)


###

##### Moyenne du prix des différents produits

In [170]:
# Moyenne des prix pour le mois de mars
filem = os.path.join("Data_Valide","mars","Data_Scrapping_mars2024.xlsx")
file_out = os.path.join("Data_Valide","mars","M_Data_Scrapping_mars2024.xlsx")
dfm = pd.read_excel(filem)
resultmars = dfm.groupby(['Code_site', 'Date_de_collecte','Libelle_du_produit','Quantite','Caracteristique','Unite','Unite_monetaire','Code produit','Code']).agg({'Prix_du_produit': 'mean'}).reset_index()
resultmars = resultmars[['Code_site', 'Date_de_collecte','Libelle_du_produit','Quantite','Prix_du_produit','Caracteristique','Unite','Unite_monetaire','Code produit','Code']]
resultmars.to_excel(file_out, index=False)

In [171]:
# Moyenne des prix pour le mois d'avril
filea = os.path.join("Data_Valide","avril","Data_Scrapping_avril2024.xlsx")
file1_out = os.path.join("Data_Valide","avril","M_Data_Scrapping_avril2024.xlsx")
dfa = pd.read_excel(filem)
resultavril = dfm.groupby(['Code_site', 'Date_de_collecte','Libelle_du_produit','Quantite','Caracteristique','Unite','Unite_monetaire','Code produit','Code']).agg({'Prix_du_produit': 'mean'}).reset_index()
resultavril = resultavril[['Code_site', 'Date_de_collecte','Libelle_du_produit','Quantite','Prix_du_produit','Caracteristique','Unite','Unite_monetaire','Code produit','Code']]
resultavril.to_excel(file1_out, index=False)

        pays  valeur   ville
0  Allemagne    30.0  Berlin
1  Allemagne    40.0  Munich
2     France    20.0    Lyon
3     France    10.0   Paris
4     Italie    50.0    Rome
